In [1]:
import numpy as np
import pandas as pd

Рассчитайте винтажную выдачу рассрочки по клиентам после реструктуризации кредита (поле date_loan) по квартальным периодам. Надо выкинуть из анализа все реструктуризации, которые были выставлены после назначения рассрочки. Рассчитайте лайфтайм реструктуризированного кредита до выдачи рассрочки

```sql
select *
from skybank.late_collection_clients a            
	join skybank.rassrochka b                
		on a.id_client = b.id_client        
where a.date_loan::date < b.date_rassrochka 
```

In [2]:
data = pd.read_csv('late_collection_rassrochka.csv')
data.drop(['cellphone', 'id_client.1'], axis=1, inplace=True)

In [3]:
data.sort_values('date_rassrochka', inplace=True)
data.head()

,id_client,id_global,id_city,gender,married,first_time,age,is_educ,is_active,cl_segm,amt_loan,date_loan,credit_type,date_rassrochka
91,2810,1084.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000000,2018-06-10,POS-1,2018-07-14
864,1771,1380.0,4.0,M,False,1.0,29.0,0.0,1.0,3.0,70000,2018-06-11,POS-1,2018-08-02
469,2900,1120.0,2.0,F,False,1.0,43.0,0.0,NaN,NaN,300000,2018-07-28,POS-1,2018-08-04
410,2925,1417.0,6.0,F,False,1.0,53.0,0.0,1.0,1.0,375000,2018-06-25,POS-1,2018-08-10
931,2362,1127.0,15.0,M,False,0.0,25.0,0.0,NaN,NaN,750000,2018-09-02,RC,2018-09-11


Приведём ячейки с датами к типу даты, создадим новый столбец в котором будет храниться разность между датой получения рассрочки и датой реструктуризации, оставим от получившейся величины только дни

In [4]:
data[['date_loan', 'date_rassrochka']] = data[['date_loan', 'date_rassrochka']].apply(pd.to_datetime, axis=1)
data['difference'] = data['date_rassrochka'].sub(data['date_loan'])
data['difference'] = data['difference'].apply(lambda x: x.days)

In [5]:
data.head()

,id_client,id_global,id_city,gender,married,first_time,age,is_educ,is_active,cl_segm,amt_loan,date_loan,credit_type,date_rassrochka,difference
91,2810,1084.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000000,2018-06-10,POS-1,2018-07-14,34
864,1771,1380.0,4.0,M,False,1.0,29.0,0.0,1.0,3.0,70000,2018-06-11,POS-1,2018-08-02,52
469,2900,1120.0,2.0,F,False,1.0,43.0,0.0,NaN,NaN,300000,2018-07-28,POS-1,2018-08-04,7
410,2925,1417.0,6.0,F,False,1.0,53.0,0.0,1.0,1.0,375000,2018-06-25,POS-1,2018-08-10,46
931,2362,1127.0,15.0,M,False,0.0,25.0,0.0,NaN,NaN,750000,2018-09-02,RC,2018-09-11,9


Забинируем поквартально столбец `difference`

In [6]:
bin_days = [0, 90, 180, 270, 360, 450, 540, 630, 720, np.inf]
labels_list = ['0-90', '91-180', '181-270', '271-360', '361-450', '451-540', '541-630', '631-720', '720+']
data['binned'] = pd.cut(data.difference, bins=bin_days, labels=labels_list)

In [7]:
grouped_by_bin = data[['id_client','binned']].groupby('binned').count()
grouped_by_bin

,id_client
binned,
0-90,102
91-180,116
181-270,118
271-360,122
361-450,96
451-540,87
541-630,81
631-720,76
720+,228


Рассчитаем лайфтайм реструктуризированного кредита до выдачи рассрочки, сгруппируем по типу кредита.

Вычислим сколько всего кредитов каждого типа было выдано:

In [8]:
total_by_type = data[['id_client', 'credit_type']].groupby('credit_type').count()

Сгруппируем выданные кредиты по типу и бину, удалим из получившейся таблицы бин `0-90`

In [9]:
grouped = data[['id_client', 'credit_type', 'binned']].groupby(['binned', 'credit_type']).count()
to_drop = [i for i in grouped.index if '0-90' in i]
grouped = grouped.drop(to_drop, axis=0)
grouped

id_client
binned  credit_type           
91-180  CC                  14
        POS-1               57
        POS-2                5
        RC                  20
        TOP-UP              20
        test                 0
181-270 CC                  11
        POS-1               50
        POS-2                4
        RC                  30
        TOP-UP              22
        test                 1
271-360 CC                  10
        POS-1               61
        POS-2                3
        RC                  21
        TOP-UP              24
        test                 3
361-450 CC                   9
        POS-1               48
        POS-2                3
        RC                  16
        TOP-UP              20
        test                 0
451-540 CC                   7
        POS-1               40
        POS-2                1
        RC                  24
        TOP-UP              14
        test                 1
541-630 CC                   8
        POS-1               43
        POS-2                1
        RC                  18
        TOP-UP              11
        test                 0
631-720 CC                   5
        POS-1               38
        POS-2                4
        RC                  19
        TOP-UP               9
        test                 1
720+    CC                  38
        POS-1               89
        POS-2                9
        RC                  54
        TOP-UP              36
        test                 2

Поскольку бинирование в pandas осуществилось несколько иначе, чем в `SQL` необходимо немного изменить таблицу `grouped`  таким образом, чтобы каждому бину и типу кредита соответствовало не количество кредитов, лежащих в интервале, а количество кредитов, чей бин больше, чем данный. Для этого развернём список с бинами, удалив из него первый бин `0-90`, и будем суммировать соответствующие ячейки таблицы `grouped` начиная с предпоследней

In [10]:
bins_ = np.flip(np.array(data.binned.unique()))
bins_ = bins_[:-1] # последний бин не нужен
for i in data.credit_type.unique():
    for j in range(1, len(bins_)):
        grouped.loc[(bins_[j], i)]['id_client'] += grouped.loc[(bins_[j-1], i)]['id_client']

In [11]:
grouped

id_client
binned  credit_type           
91-180  CC                 102
        POS-1              426
        POS-2               30
        RC                 202
        TOP-UP             156
        test                 8
181-270 CC                  88
        POS-1              369
        POS-2               25
        RC                 182
        TOP-UP             136
        test                 8
271-360 CC                  77
        POS-1              319
        POS-2               21
        RC                 152
        TOP-UP             114
        test                 7
361-450 CC                  67
        POS-1              258
        POS-2               18
        RC                 131
        TOP-UP              90
        test                 4
451-540 CC                  58
        POS-1              210
        POS-2               15
        RC                 115
        TOP-UP              70
        test                 4
541-630 CC                  51
        POS-1              170
        POS-2               14
        RC                  91
        TOP-UP              56
        test                 3
631-720 CC                  43
        POS-1              127
        POS-2               13
        RC                  73
        TOP-UP              45
        test                 3
720+    CC                  38
        POS-1               89
        POS-2                9
        RC                  54
        TOP-UP              36
        test                 2

In [12]:
ltv = {}
for i in data.credit_type.unique():
    ltv[i] = 0
    for j in bins_:
        if j in ('91-180', '720+'):
            ltv[i] += (grouped.loc[(j, i)]['id_client']) / (2 * total_by_type.loc[i]['id_client'])
        else:
            ltv[i] += grouped.loc[(j, i)]['id_client'] /  total_by_type.loc[i]['id_client']

Итоговый LTV по каждому типу кредита представлен в таблице ниже.

In [13]:
types = pd.Series(ltv.keys(), name='credit_type')
ltvs = pd.Series(ltv.values(), name='LTV')
res = pd.concat([types, ltvs], axis=1)
res

,credit_type,LTV
0,POS-1,3.563542
1,RC,3.927928
2,TOP-UP,3.613095
3,POS-2,3.585714
4,CC,4.017699
5,test,4.250000
